# 🌱 Setup Ambiente - Gerador de Pastagens Brasileiras

Este notebook configura o ambiente completo para geração de imagens sintéticas de pastagens brasileiras usando Stable Diffusion.

**Características:**
- ✅ Otimizado para Google Colab (GPU T4/V100)
- ✅ Instalação automática de dependências
- ✅ Verificação de GPU e recursos
- ✅ Download de modelos base
- ✅ Configuração de cache otimizado


## 🔧 1. Verificação de Sistema

In [ ]:
import sys
import torch
import platform
import psutil
from pathlib import Path

print("🖥️  INFORMAÇÕES DO SISTEMA")
print("=" * 50)
print(f"Python: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"CPU cores: {psutil.cpu_count()}")
print(f"RAM total: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print(f"RAM disponível: {psutil.virtual_memory().available / (1024**3):.1f} GB")

print("\n🚀 INFORMAÇÕES GPU")
print("=" * 50)
if torch.cuda.is_available():
    print(f"✅ CUDA disponível: {torch.version.cuda}")
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ Memória GPU: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")
    print(f"✅ Compute Capability: {torch.cuda.get_device_capability(0)}")
else:
    print("❌ CUDA não disponível - funcionará apenas em CPU")
    print("⚠️  Para melhor performance, use runtime GPU no Colab")

print("\n📁 DIRETÓRIOS COLAB")
print("=" * 50)
print(f"Working dir: {Path.cwd()}")
if Path('/content').exists():
    print("✅ Executando no Google Colab")
    print(f"Espaço disponível: {psutil.disk_usage('/content').free / (1024**3):.1f} GB")
else:
    print("⚠️  Não detectado Google Colab - ajuste paths se necessário")

## 📥 2. Clone do Repositório

In [ ]:
import os
from pathlib import Path

# Verificar se já está clonado
if Path('brazilian-pasture-synthesis').exists():
    print("✅ Repositório já existe - atualizando...")
    %cd brazilian-pasture-synthesis
    !git pull origin main
else:
    print("📥 Clonando repositório...")
    # SUBSTITUA pela URL real do seu repositório
    !git clone https://github.com/seu-usuario/brazilian-pasture-synthesis.git
    %cd brazilian-pasture-synthesis

# Verificar estrutura
print("\n📂 ESTRUTURA DO PROJETO:")
!find . -maxdepth 2 -type d | head -20

## 🔧 3. Instalação de Dependências

In [ ]:
# Instalar dependências do requirements.txt
print("📦 Instalando dependências principais...")
!pip install -r requirements.txt

print("\n🔧 Instalando dependências específicas do Colab...")
# xFormers para otimização de memória
!pip install xformers==0.0.22.post7 --index-url https://download.pytorch.org/whl/cu118

# Dependências de qualidade de imagem
!pip install lpips pytorch-fid

print("\n✅ Instalação concluída!")

## 📁 4. Configuração de Diretórios

In [ ]:
import os
from pathlib import Path

# Diretórios essenciais para Colab
directories = [
    "/content/model_cache",
    "/content/generated_cache", 
    "/content/outputs",
    "/content/datasets",
    "/content/temp",
    "outputs/generated_images",
    "outputs/datasets", 
    "outputs/models",
    "outputs/evaluations",
    "outputs/samples"
]

print("📁 Criando estrutura de diretórios...")
for directory in directories:
    Path(directory).mkdir(parents=True, exist_ok=True)
    print(f"✅ {directory}")

print("\n📊 Espaço em disco:")
if Path('/content').exists():
    disk_usage = psutil.disk_usage('/content')
    print(f"Total: {disk_usage.total / (1024**3):.1f} GB")
    print(f"Usado: {disk_usage.used / (1024**3):.1f} GB")
    print(f"Livre: {disk_usage.free / (1024**3):.1f} GB")


## ⚙️ 5. Configuração de Variáveis de Ambiente

In [ ]:
import os

# Configurações de otimização de memória
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Diretórios de cache
os.environ['HF_HOME'] = '/content/model_cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/model_cache'
os.environ['DIFFUSERS_CACHE'] = '/content/model_cache'

# Configurações específicas
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['OMP_NUM_THREADS'] = '1'

print("⚙️ Variáveis de ambiente configuradas:")
important_vars = [
    'PYTORCH_CUDA_ALLOC_CONF', 'HF_HOME', 'TRANSFORMERS_CACHE', 
    'DIFFUSERS_CACHE', 'CUDA_VISIBLE_DEVICES'
]

for var in important_vars:
    print(f"✅ {var}: {os.environ.get(var, 'Not set')}")

## 🧪 6. Teste de Importações

In [ ]:
import sys
print("🧪 Testando importações principais...")
print("=" * 50)

# Lista de importações críticas
imports_to_test = [
    ('torch', 'PyTorch'),
    ('torchvision', 'TorchVision'),
    ('diffusers', 'Hugging Face Diffusers'),
    ('transformers', 'Hugging Face Transformers'), 
    ('accelerate', 'Accelerate'),
    ('controlnet_aux', 'ControlNet Auxiliary'),
    ('ultralytics', 'YOLOv8/v9'),
    ('cv2', 'OpenCV'),
    ('PIL', 'Pillow'),
    ('numpy', 'NumPy'),
    ('matplotlib', 'Matplotlib'),
    ('yaml', 'PyYAML'),
    ('albumentations', 'Albumentations'),
    ('tqdm', 'TQDM')
]

failed_imports = []

for module, name in imports_to_test:
    try:
        __import__(module)
        print(f"✅ {name:25} - OK")
    except ImportError as e:
        print(f"❌ {name:25} - FAILED: {e}")
        failed_imports.append(module)

# Testar importações específicas do projeto
print("\n🔧 Testando módulos do projeto...")
print("=" * 50)

project_imports = [
    ('src.diffusion.pipeline_manager', 'Pipeline Manager'),
    ('src.diffusion.prompt_engine', 'Prompt Engine'), 
    ('src.dataset.generator', 'Dataset Generator'),
    ('src.dataset.quality_metrics', 'Quality Metrics')
]

for module, name in project_imports:
    try:
        __import__(module)
        print(f"✅ {name:25} - OK")
    except ImportError as e:
        print(f"❌ {name:25} - FAILED: {e}")
        failed_imports.append(module)

# Resumo
if failed_imports:
    print(f"\n⚠️ {len(failed_imports)} importações falharam:")
    for module in failed_imports:
        print(f"   - {module}")
    print("\n🔧 Execute a célula de instalação novamente se necessário")
else:
    print("\n🎉 Todas as importações foram bem-sucedidas!")
    print("✅ Sistema pronto para uso")

## 🚀 7. Teste de GPU

In [ ]:
import torch
import time

if torch.cuda.is_available():
    print("🧪 Testando performance da GPU...")
    print("=" * 50)
    
    device = torch.cuda.current_device()
    
    # Teste básico de alocação
    print("🔸 Teste 1: Alocação de memória")
    try:
        test_tensor = torch.randn(1000, 1000).cuda()
        print("✅ Alocação básica: OK")
        del test_tensor
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ Erro na alocação: {e}")
    
    # Teste de operações
    print("\n🔸 Teste 2: Operações matemáticas")
    try:
        start_time = time.time()
        a = torch.randn(2000, 2000).cuda()
        b = torch.randn(2000, 2000).cuda() 
        c = torch.mm(a, b)
        torch.cuda.synchronize()
        end_time = time.time()
        
        print(f"✅ Multiplicação de matrizes: {end_time - start_time:.3f}s")
        
        del a, b, c
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"❌ Erro nas operações: {e}")
    
    # Informações de memória
    print("\n🔸 Status da memória GPU:")
    print(f"   Alocada: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"   Reservada: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
    print(f"   Máximo alocado: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")
    
    # Reset stats
    torch.cuda.reset_peak_memory_stats()
    
    print("\n✅ GPU funcionando corretamente!")
    
else:
    print("⚠️ GPU não disponível - funcionará em CPU")
    print("Para melhor performance, ative GPU no Colab:")
    print("Runtime > Change runtime type > GPU")

## 📥 8. Download Rápido de Modelos (Opcional)

In [ ]:
# Esta célula faz download dos modelos principais - pode ser demorada
# Execute apenas se quiser fazer cache dos modelos

from diffusers import StableDiffusionXLPipeline, ControlNetModel
import torch

download_models = input("Fazer download dos modelos agora? (s/N): ")

if download_models.lower() in ['s', 'sim', 'y', 'yes']:
    print("📥 Fazendo download dos modelos base...")
    print("⏳ Isso pode demorar alguns minutos...")
    
    try:
        # Modelo principal Stable Diffusion XL
        print("\n🔸 Baixando Stable Diffusion XL...")
        model_id = "stabilityai/stable-diffusion-xl-base-1.0"
        pipe = StableDiffusionXLPipeline.from_pretrained(
            model_id,
            cache_dir="/content/model_cache",
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        )
        print("✅ Stable Diffusion XL baixado com sucesso!")
        del pipe  # Liberar memória
        
        # Modelo ControlNet
        print("\n🔸 Baixando ControlNet Canny...")
        controlnet = ControlNetModel.from_pretrained(
            "lllyasviel/sd-controlnet-canny",
            cache_dir="/content/model_cache",
            torch_dtype=torch.float16
        )
        print("✅ ControlNet Canny baixado com sucesso!")
        del controlnet
        
        # Limpar cache GPU
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            
        print("\n🎉 Todos os modelos baixados com sucesso!")
        print("💾 Modelos salvos em: /content/model_cache")
        
    except Exception as e:
        print(f"❌ Erro no download: {e}")
        print("⚠️ Modelos serão baixados conforme necessário")
        
else:
    print("⏩ Pulando download - modelos serão baixados conforme necessário")
    print("💡 Isso pode causar delay na primeira execução")

## ✅ 9. Verificação Final

In [ ]:
import os
from pathlib import Path
import torch

print("🔍 VERIFICAÇÃO FINAL DO SETUP")
print("=" * 50)

# Verificar instalação
checks = [
    ("Python >= 3.8", sys.version_info >= (3, 8)),
    ("PyTorch instalado", 'torch' in sys.modules),
    ("Diffusers instalado", 'diffusers' in sys.modules),
    ("CUDA disponível", torch.cuda.is_available()),
    ("Diretórios criados", Path('/content/model_cache').exists()),
    ("Projeto encontrado", Path('src').exists()),
    ("Configs encontrados", Path('configs').exists())
]

all_ok = True
for check_name, result in checks:
    status = "✅" if result else "❌"
    print(f"{status} {check_name}")
    if not result:
        all_ok = False

print("\n" + "=" * 50)
if all_ok:
    print("🎉 SETUP COMPLETO COM SUCESSO!")
    print("")
    print("📚 PRÓXIMOS PASSOS:")
    print("1. Execute '01_Explore_Prompts.ipynb' para testar prompts")
    print("2. Use '02_Generate_Dataset.ipynb' para gerar imagens")
    print("3. Execute '03_Quality_Control.ipynb' para análise de qualidade")
    print("")
    print("💡 DICA: Salve uma cópia deste notebook no seu Drive!")
else:
    print("⚠️ SETUP INCOMPLETO")
    print("Revise os passos anteriores e corrija os problemas")
    print("Em caso de dúvidas, consulte a documentação")

print(f"\n🕐 Setup concluído em: {time.strftime('%Y-%m-%d %H:%M:%S')}")

## 🛠️ Troubleshooting

### Problemas Comuns:

**1. GPU não detectada:**
- Vá em `Runtime > Change runtime type > GPU`
- Reinicie o runtime

**2. Erro de memória:**
- Reinicie o runtime: `Runtime > Restart runtime`
- Reduza batch_size nos experimentos

**3. Erro de importação:**
- Execute novamente a célula de instalação
- Verifique se há conflitos de versões

**4. Download lento:**
- Use servidor do Colab mais próximo
- Execute durante horários de menor uso

### Comandos Úteis:
```python
# Verificar uso de GPU
!nvidia-smi

# Limpar cache
torch.cuda.empty_cache()

# Verificar espaço em disco
!df -h

# Listar processos GPU
!fuser -v /dev/nvidia*
```